# Deep learning model Training:

## Importing Data:

In [2]:
import cv2
import os
features=[]
targets=[]
path="D:/train/"
folders={"angry":0,"disgust":1,"fear":2,"happy":3,"neutral":4,"sad":5,"surprise":6}
for i in folders:
    C=os.listdir(path+str(i))
    for j in C:
        image=cv2.imread(path+str(i)+"/"+str(j))
        try:
            image=cv2.resize(image,(32,32))
        except:
            pass
        else:
            features.append(image)
            targets.append(folders[i])
    print(i)

angry
disgust
fear
happy
neutral
sad
surprise


In [23]:
import numpy as np
targets=np.array(targets)
features=np.array(features)

In [24]:
features.shape

(28821, 32, 32, 3)

In [25]:
targets.shape

(28821,)

In [26]:
(train_features,train_target)=(features,targets)

## Converting Data into required format:

In [27]:
def preprocessing(image):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=image/255
    return image

In [28]:
train_features.shape

(28821, 32, 32, 3)

In [29]:
train_features=np.array(list(map(preprocessing,train_features)))


In [30]:
train_features=train_features.reshape(28821, 32, 32, 1)

In [31]:
train_features.shape

(28821, 32, 32, 1)

In [32]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rotation_range=1,width_shift_range=0,height_shift_range=0,zoom_range=0,shear_range=0)

## Converting targets to categorical:

In [33]:
datagen.fit(train_features)

In [34]:
from tensorflow.keras.utils import to_categorical

In [35]:
train_target=to_categorical(train_target)

## Specifying Model:

In [5]:
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential

Model = Sequential()
Model.add(Conv2D(64,(3,3), padding='same', input_shape=(32, 32,1)))
Model.add(BatchNormalization())
Model.add(Activation('relu'))
Model.add(MaxPooling2D((2, 2)))
Model.add(Dropout(0.25))
Model.add(Conv2D(128,(5,5), padding='same'))
Model.add(BatchNormalization())
Model.add(Activation('relu'))
Model.add(MaxPooling2D((2, 2)))
Model.add(Dropout(0.25))
Model.add(Conv2D(512,(3,3), padding='same'))
Model.add(BatchNormalization())
Model.add(Activation('relu'))
Model.add(MaxPooling2D((2, 2)))
Model.add(Dropout(0.25))
Model.add(Conv2D(512,(3,3), padding='same'))
Model.add(BatchNormalization())
Model.add(Activation('relu'))
Model.add(MaxPooling2D((2, 2)))
Model.add(Dropout(0.25))
Model.add(Flatten())
Model.add(Dense(256))
Model.add(BatchNormalization())
Model.add(Activation('relu'))
Model.add(Dropout(0.5))
Model.add(Dense(512))
Model.add(BatchNormalization())
Model.add(Activation('relu'))
Model.add(Dropout(0.5))
Model.add(Dense(14, activation='relu'))
Model.add(Dense(7, activation='softmax'))


## Compiling Model:

In [6]:
from keras.optimizers import Adam

Model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

## Loading previous weights:
#### ( Model is trained multiple times so that to run PC for less time continously and achive better accuracy)

In [7]:
Model.load_weights("AI_April-2021_SourabhSanganwar_facemodelweights.h5")

## Training model:
##### ( I had trained it many times in parts and loaded weights now iam training it for one epoch for illustration)

In [39]:
Model.fit_generator(datagen.flow(train_features,train_target,batch_size=20),epochs=20)

1442/1442 [==============================] - 267s 184ms/step - loss: 0.2895 - accuracy: 0.9009


## Saving model to file :

In [40]:
model_2=Model.to_json()
path="D:/modelData/facemodelarchitecture.json"
file_model=open(path,"w")
file_model.write(model_2)
file_model.close()
Model.save_weights("D:/modelData/facemodelweights.h5")

# Finding emotions using webcam

## Loading Model:

In [2]:
import keras

from keras.models import model_from_json
path="AI_April-2021_SourabhSanganwar_facemodelarchitecture.json"
modelfile=open(path,'r')
Model=model_from_json(modelfile.read())
modelfile.close()
Model.load_weights("AI_April-2021_SourabhSanganwar_facemodelweights.h5")

In [3]:
import numpy as np
import cv2

## setting up input from webcam:

In [4]:
brightness=180
capt=cv2.VideoCapture(0)
capt.set(3,640)
capt.set(4,480)
capt.set(10,brightness)

True

## defining functions and maps in prediction:

In [5]:
def preprocess(image):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)           
    image=image/255                                        
    return image
emotions={0:"Angry",1:"Disgust",2:"Fear",3:"Happy",4:"Neutral",5:"Sad",6:"Surprise"}

def max_index (a,b):
    for i in range(0,7):
        if a[0][i]==b:
            return i
colour={"Angry":(0,0,250),"Disgust":(0,255,255),"Fear":(255,255,0),"Happy":(0,255,0),"Neutral":(0,0,0),"Sad":(200,0,129),"Surprise":(129,64,18)}

## Predicting and showing image in webcam:

In [8]:
# final
while True:
    message,images=capt.read()
    images=cv2.flip(images,1,0)
    temp=np.asarray(images)
    image=np.asarray(images)
    face_cascade=cv2.CascadeClassifier("haarcascade.xml")
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(image,1.1,4)
    proba=0
    Emotion=" "
    Asc=0
    for (x,y,w,h) in faces:
        imag=images[x:x+w,y:y+h]
        try:
            imag=cv2.resize(imag,(32,32))
        except:
            pass
        else:
            imag=preprocess(imag)
            imag=imag.reshape(1,32,32,1)
            lists=Model.predict(imag)
            proba=lists[0]
            proba=proba.max()
            index=0
            for i in range(0,7):
                if lists[0][i]==proba:
                    index=i
                    break
            Emotion=emotions[max_index(lists,proba)]
        cv2.putText(images,"Facial Expression Detector",(100,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,250,0),2)
        if proba>0.6:
            cv2.putText(images,Emotion,(x+int(2*w/200),y+int(30*w/200)),cv2.FONT_HERSHEY_SIMPLEX,1*w/200,colour[Emotion],2*w//300+1)
        images=cv2.rectangle(images,(x,y),(x+w,y+h),(255,0,0),1)
    cv2.putText(images,"Facial Expression Detector",(100,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,250,0),2)
    cv2.imshow("ModelPrediction",images)
    Asc=cv2.waitKey(20)
    if Asc==ord("e"):
        cv2.destroyAllWindows()
        break
    